### Teoría sobre la codificación de Huffman

El **algoritmo de Huffman** es uno de los métodos más conocidos para la compresión de datos sin pérdidas. 
A grandes rasgos, se basa en:

1. **Calcular la frecuencia** de aparición de cada carácter en el texto de entrada.
2. **Crear nodos hoja** para cada carácter, etiquetados con su frecuencia.
3. **Usar una cola de prioridad (min-heap)** para ir combinando de abajo hacia arriba los nodos de menor frecuencia:
   - Extraemos los dos nodos de menor frecuencia.
   - Creamos un nodo padre que los agrupe, con frecuencia igual a la suma de ambos.
   - Insertamos de nuevo este nodo padre en la cola de prioridad.
4. Al finalizar, **queda un solo nodo** en la cola, que es la raíz de nuestro árbol de Huffman.
5. Para **codificar** cada carácter, se recorre el árbol desde la raíz hasta la hoja correspondiente:
   - Se asigna `0` al ir por la rama izquierda.
   - Se asigna `1` al ir por la rama derecha.
   - La concatenación de esos bits es el **código Huffman** para dicho carácter.

Este método produce **códigos más cortos** para los caracteres más frecuentes y **códigos más largos** para los menos frecuentes, logrando una compresión eficiente en muchos casos.



#### Pseudocódigo del algoritmo de Huffman

Un código de Huffman es un árbol, construido de abajo hacia arriba, comenzando con la lista de diferentes caracteres que aparecen en un texto y su frecuencia. El algoritmo itera de la siguiente manera: 

- Selecciona y remueve de la lista los dos elementos con la frecuencia más baja.
- Luego crea un nuevo nodo combinándolos (sumando las dos frecuencias).
- Y finalmente agrega de nuevo el nuevo nodo a la lista. 

Aunque el árbol en sí no es un heap, un paso clave del algoritmo se basa en recuperar de forma eficiente los elementos más pequeños de la lista, así como en agregar nuevos elementos a la lista de manera eficiente.


```
function huffman(text) 
  charFrequenciesMap ← ComputeFrequencies(text) 
  priorityQueue ← MinHeap() 
  for (char, frequency) in charFrequenciesMap do 
    priorityQueue.insert(TreeNode([char], frequency)) 
  while priorityQueue.size > 1 do 
    left ← priorityQueue.top() 
    right ← priorityQueue.top() 
    parent ← TreeNode(left.chars + right.chars, 
                       left.frequency + right.frequency) 
    parent.left ← left 
    parent.right ← right 
    priorityQueue.insert(parent) 
  return buildTable(priorityQueue.top(), [], Map()) 
```

> Cada `TreeNode`, de hecho, contiene dos campos (además de los punteros a sus hijos):  
> - un **conjunto de caracteres**, y  
> - la **frecuencia** de esos caracteres en el texto, calculada como la suma de las frecuencias de los caracteres individuales.

**Construyendo una tabla a partir del árbol**

```
function buildTable(node, sequence, charactersToSequenceMap) 
  if node.characters.size == 1 then 
    charactersToSequenceMap[node.characters[0]] ← sequence  
  else 
    if node.left <> null then 
      buildTable(node.left, 0 + sequence, charactersToSequenceMap)  
    if node.right <> null then 
      buildTable(node.right, 1 + sequence, charactersToSequenceMap)  
  return charactersToSequenceMap 
```

> Estos pasos se repiten hasta que quede un solo elemento en la cola  y ese último elemento será el `TreeNode` que representa la **raíz del árbol final**.

> Escribimos el método `buildTable` en forma **recursiva**.  Esto nos permite ofrecer un código más **limpio y fácil de entender**,  pero en algunos lenguajes las implementaciones concretas pueden ser más eficientes  si se implementan usando **iteraciones explícitas**.


### Implementaciones

In [ ]:
from typing import Any, List, Optional, Tuple


class DWayHeap(object):
    def __init__(self, elements: List[Any] = [], priorities: List[float] = [], branching_factor: int = 2) -> None:
        """Constructor

        Args:
            elements: Los elementos para inicializar el heap.
            priorities: Las prioridades de los elementos anteriores. Deben tener la misma longitud que `elements`.
            branching_factor: El número (máximo) de hijos por nodo en el heap. Debe ser al menos 2.
        """
        if len(elements) != len(priorities):
            raise ValueError(f'La longitud de la lista de elementos ({len(elements)})'
                             f' debe coincidir con la longitud de la lista de prioridades ({len(priorities)}).')
        if branching_factor < 2:
            raise ValueError(f'El factor de ramificación ({branching_factor}) debe ser mayor que 1.')
        self._pairs: List[Tuple[float, Any]] = []
        self.D = branching_factor

        if len(elements) > 0:
            self._heapify(elements, priorities)

    def __sizeof__(self) -> int:
        """Tamaño del heap.

        Returns: El número de elementos en el heap.
        """
        return len(self)

    def __len__(self) -> int:
        """Tamaño del heap.

        Returns: El número de elementos en el heap.
        """
        return len(self._pairs)

    def _validate(self) -> bool:
        """Verifica que se cumplan las tres invariantes del heap:
        1. Cada nodo tiene como máximo `D` hijos. (Garantizado por la construcción)
        2. El árbol del heap es completo y alineado a la izquierda. (También garantizado por la construcción)
        3. Cada nodo contiene la mayor prioridad en el subárbol con raíz en ese nodo.

        Returns: True si se cumplen todas las invariantes del heap.
        """
        current_index = 0
        first_leaf = self.first_leaf_index()
        while current_index < first_leaf:
            current_priority: float = self._pairs[current_index][0]
            first_child = self._first_child_index(current_index)
            last_child_guard = min(first_child + self.D, len(self))
            for child_index in range(first_child, last_child_guard):
                if current_priority < self._pairs[child_index][0]:
                    return False
            current_index += 1
        return True

    def _push_down(self, index: int) -> None:
        """Empuja hacia abajo la raíz de un sub-heap hacia sus hojas para restablecer las invariantes del heap.
        Si alguno de los hijos del elemento tiene mayor prioridad, se intercambia el elemento actual
        con su hijo de mayor prioridad C, y se verifica recursivamente el sub-heap que estaba previamente
        enraizado en ese C.

        Args:
            index: El índice de la raíz del sub-heap.
        """
        assert (0 <= index < len(self._pairs))
        input_pair = self._pairs[index]
        input_priority = input_pair[0]
        current_index = index
        first_leaf = self.first_leaf_index()
        while current_index < first_leaf:
            child_index = self._highest_priority_child_index(current_index)
            assert (child_index is not None)
            if self._pairs[child_index][0] > input_priority:
                self._pairs[current_index] = self._pairs[child_index]
                current_index = child_index
            else:
                break

        self._pairs[current_index] = input_pair

    def _bubble_up(self, index: int) -> None:
        """Eleva un elemento hacia la raíz para restablecer las invariantes del heap.
        Si el padre P de un elemento tiene menor prioridad, se intercambian el elemento actual y su padre,
        y se verifica recursivamente la posición que tenía P anteriormente.

        Args:
            index: El índice del elemento a elevar.
        """
        assert (0 <= index < len(self._pairs))
        input_pair = self._pairs[index]
        input_priority = input_pair[0]
        while index > 0:
            parent_index = self._parent_index(index)
            parent = self._pairs[parent_index]

            if input_priority > parent[0]:
                self._pairs[index] = parent
                index = parent_index
            else:
                break

        self._pairs[index] = input_pair

    def _first_child_index(self, index) -> int:
        """Calcula el índice del primer hijo de un nodo en el heap.

        Args:
            index: El índice del nodo actual, del cual se buscan los índices de sus hijos.

        Returns: El índice del hijo más a la izquierda del nodo actual del heap.
        """
        return index * self.D + 1

    def _parent_index(self, index) -> int:
        """Calcula el índice del padre de un nodo en el heap.

        Args:
            index: El índice del nodo actual, del cual se busca el índice de su padre.

        Returns: El índice del padre del nodo actual del heap.
        """
        return (index - 1) // self.D

    def _highest_priority_child_index(self, index) -> Optional[int]:
        """Encuentra, entre los hijos de un nodo del heap, el hijo con la mayor prioridad.
        En caso de que varios hijos tengan la misma prioridad, se devuelve el más a la izquierda.

        Args:
            index: El índice del nodo del heap cuyos hijos se examinan.

        Returns: El índice del hijo con mayor prioridad del nodo actual del heap, o None si
                 el nodo actual no tiene hijos.
        """
        first_index = self._first_child_index(index)
        size = len(self)
        last_index = min(first_index + self.D, size)

        if first_index >= size:
            return None

        highest_priority = -float('inf')
        index = first_index
        for i in range(first_index, last_index):
            if self._pairs[i][0] > highest_priority:
                highest_priority = self._pairs[i][0]
                index = i

        return index

    def first_leaf_index(self):
        return (len(self) - 2) // self.D + 1

    def _heapify(self, elements: List[Any], priorities: List[float]) -> None:
        """Inicializa el heap con una lista de elementos y prioridades.

        Args:
            elements: La lista de elementos a agregar al heap.
            priorities: Las prioridades correspondientes a esos elementos (en el mismo orden).
        """
        assert (len(elements) == len(priorities))
        self._pairs = list(zip(priorities, elements))
        last_inner_node_index = self.first_leaf_index() - 1
        for index in range(last_inner_node_index, -1, -1):
            self._push_down(index)

    def is_empty(self) -> bool:
        """Verifica si el heap está vacío.

        Returns: True si el heap está vacío.
        """
        return len(self) == 0

    def top(self) -> Any:
        """Elimina y devuelve el elemento con mayor prioridad en el heap.
        Si el heap está vacío, lanza una `RuntimeError`.

        Returns: El elemento con mayor prioridad en el heap.
        """
        if self.is_empty():
            raise RuntimeError('Se llamó al método top en un heap vacío.')
        if len(self) == 1:
            element = self._pairs.pop()[1]
        else:
            element = self._pairs[0][1]
            self._pairs[0] = self._pairs.pop()
            self._push_down(0)

        return element

    def peek(self) -> Any:
        """Devuelve, SIN eliminarlo, el elemento con mayor prioridad en el heap.
        Si el heap está vacío, lanza una `RuntimeError`.

        Returns: El elemento con mayor prioridad en el heap.
        """
        if self.is_empty():
            raise RuntimeError('Se llamó al método peek en un heap vacío.')
        return self._pairs[0][1]

    def insert(self, element: Any, priority: float) -> None:
        """Agrega un nuevo par elemento/prioridad al heap

        Args:
            element: El nuevo elemento a agregar.
            priority: La prioridad asociada al nuevo elemento.
        """
        self._pairs.append((priority, element))
        self._bubble_up(len(self._pairs) - 1)


####  HuffmanNode y funciones asociadas 

In [ ]:
import collections
from typing import Dict, List, Optional

class HuffmanNode(object):
    def __init__(self, symbols: List[str], priority: float, left: Optional['HuffmanNode'] = None,
                 right: Optional['HuffmanNode'] = None) -> None:
        """Constructor de un nodo para el árbol de Huffman.
        
        Args:
            symbols: Lista de símbolos (caracteres) contenidos en el nodo.
            priority: Valor de prioridad (frecuencia) asociado al nodo.
            left: Nodo hijo izquierdo (opcional).
            right: Nodo hijo derecho (opcional).
        """
        self._symbols = symbols
        self._priority = priority
        self._left = left
        self._right = right

    def __repr__(self):
        """Representación oficial del nodo."""
        return f'({self._symbols}, {self._priority})'

    def __str__(self) -> str:
        """Representación en cadena del nodo, mostrando sus hijos."""
        return f'{repr(self)} -> ({self._left} | {self._right})'

    def symbols(self) -> List[str]:
        """Devuelve la lista de símbolos contenidos en el nodo.
        
        Returns:
            Lista de símbolos.
        """
        return self._symbols

    def priority(self) -> float:
        """Devuelve la prioridad (frecuencia) del nodo.
        
        Returns:
            Valor de prioridad.
        """
        return self._priority

    @staticmethod
    def encode_left_path(inner_path: str) -> str:
        """Codifica la ruta agregando un 0 al inicio (para la rama izquierda).
        
        Args:
            inner_path: La codificación interna.
        
        Returns:
            La ruta codificada para la rama izquierda.
        """
        return f'0{inner_path}'

    @staticmethod
    def encode_right_path(inner_path: str) -> str:
        """Codifica la ruta agregando un 1 al inicio (para la rama derecha).
        
        Args:
            inner_path: La codificación interna.
        
        Returns:
            La ruta codificada para la rama derecha.
        """
        return f'1{inner_path}'

    def _validate(self) -> bool:
        """Valida la consistencia del nodo verificando que:
           - La lista de símbolos sea la concatenación de la de sus hijos.
           - La prioridad sea la suma de las prioridades de sus hijos.
        
        Returns:
            True si el nodo es válido, False en caso contrario.
        """
        left_symbols = self._left.symbols() if self._left else []
        right_symbols = self._right.symbols() if self._right else []

        left_priority = self._left.priority() if self._left else 0.
        right_priority = self._right.priority() if self._right else 0.

        if self.symbols() != left_symbols + right_symbols:
            return False

        if self.priority() != left_priority + right_priority:
            return False

        return True

    def tree_encoding(self) -> Dict[str, str]:
        """Genera la codificación del árbol de Huffman.
        
        Returns:
            Un diccionario que asocia cada símbolo a su cadena de codificación.
        """
        left_encoding_table = {} if self._left is None else self._left.tree_encoding()
        right_encoding_table = {} if self._right is None else self._right.tree_encoding()

        encoding_table = {}

        for key, path in left_encoding_table.items():
            encoding_table[key] = HuffmanNode.encode_left_path(path)

        for key, path in right_encoding_table.items():
            encoding_table[key] = HuffmanNode.encode_right_path(path)

        if len(self._symbols) == 1:
            encoding_table[self.symbols()[0]] = ""

        return encoding_table

def _create_frequency_table(text: str) -> collections.Counter:
    """Dado un texto, crea una tabla de frecuencias que asocia cada caracter a su número de ocurrencias.
    
    Args:
        text: Texto de entrada.
    
    Returns:
        Un objeto Counter con la frecuencia de cada caracter.
    """
    return collections.Counter(text)

def _frequency_table_to_heap(ft: collections.Counter, branching_factor: int = 2) -> DWayHeap:
    """Convierte una tabla de frecuencias en un heap cuyos elementos son nodos del árbol de Huffman.
    
    Args:
        ft: Tabla de frecuencias (caracter: número de ocurrencias).
        branching_factor: Factor de ramificación para el heap d-ario.
    
    Returns:
        Un heap d-ario que contiene un nodo por cada carácter único en el texto.
    """
    characters, priorities = list(zip(*ft.items()))
    # Crea un nodo para cada carácter; se utiliza el inverso de la frecuencia ya que DWayHeap es un heap máximo
    priorities = list(map(lambda p: -p, priorities))
    elements = list(map(lambda c: HuffmanNode([c], -ft[c]), characters))
    return DWayHeap(elements=elements, priorities=priorities, branching_factor=branching_factor)

def _heap_to_tree(heap: DWayHeap) -> HuffmanNode:
    """Construye el árbol de codificación de Huffman a partir de un heap.
    
    Args:
        heap: Un heap d-ario que contiene nodos de Huffman.
    
    Returns:
        El nodo raíz del árbol de codificación de Huffman.
    """
    while len(heap) > 1:
        # Obtiene los dos nodos con mayor prioridad
        right: HuffmanNode = heap.top()
        left: HuffmanNode = heap.top()

        # Combina los símbolos y las prioridades de los nodos extraídos
        symbols: List[str] = left.symbols() + right.symbols()
        priority: float = left.priority() + right.priority()

        heap.insert(HuffmanNode(symbols, priority, left, right), priority)

    return heap.top()

def create_encoding(text: str, branching_factor: int) -> Dict[str, str]:
    """Crea la codificación de Huffman para un texto.
    
    Args:
        text: Texto de entrada a comprimir.
        branching_factor: Factor de ramificación para el heap d-ario.
    
    Returns:
        Un diccionario que asocia cada carácter único del texto con su codificación binaria.
        Por ejemplo, si ('a', '101') está en el diccionario, para comprimir el texto se reemplaza 'a' por '101'.
    """
    return _heap_to_tree(_frequency_table_to_heap(_create_frequency_table(text), branching_factor)).tree_encoding()


#### Pruebas

In [ ]:
import unittest

class HuffmanTest(unittest.TestCase):
    Text = "fffeeeeeddddddcccccccbbbbbbbbbbbbbbbbbbbbbbaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

    def test_huffman(self):
        self.assertEqual({'a': '0', 'b': '10', 'c': '1100', 'd': '1101', 'e': '1110', 'f': '1111'},
                         create_encoding(HuffmanTest.Text, 2))

    def test_create_frequency_table(self):
        self.assertEqual({'a': 57, 'b': 22, 'c': 7, 'd': 6, 'e': 5, 'f': 3},
                         _create_frequency_table(HuffmanTest.Text))

    def test_frequency_table_to_heap(self):
        heap = _frequency_table_to_heap(_create_frequency_table(HuffmanTest.Text))
        self.assertTrue(heap._validate())

    def test_heap_to_tree(self):
        heap = _frequency_table_to_heap(_create_frequency_table(HuffmanTest.Text))
        tree = _heap_to_tree(heap)
        self.assertTrue(tree._validate())
        print(tree)

# Ejecuta las pruebas
suite = unittest.TestLoader().loadTestsFromTestCase(HuffmanTest)
unittest.TextTestRunner(verbosity=2).run(suite)


#### Profiling

In [ ]:
import base64
import cProfile
import pstats
import unittest

from typing import List, Tuple

def read_text(file_name: str) -> str:
    with open(file_name, 'r', encoding='utf-8') as f:
        texto = f.read()
    return texto


def read_image(file_name: str) -> str:
    with open(file_name, 'rb') as f:
        # Lee los bytes de una imagen y los codifica en base64 para tratarlos como texto
        texto = str(base64.b64encode(f.read()))
    return texto


# Clase para realizar el perfilado de la codificación de Huffman
class HuffmanProfile(unittest.TestCase):
    TestCases = {
        'texto': (['data/alice.txt', 'data/candide.txt',
                   'data/gullivers_travels.txt'], read_text, 1000),
        'image': (['data/best_advice.jpg'], read_image, 200)
    }
    BranchingFactors = range(2, 24)
    OutputFileName = 'data/stats_huffman.csv'

    @staticmethod
    def write_header(f) -> None:
        f.write('test_case,branching_factor,method_name,total_time,cumulative_time,per_call_time\n')

    @staticmethod
    def write_row(f, test_case: str, branching_factor: int, method_name: str, total_time: float,
                  cumulative_time: float, per_call_time: float) -> None:
        f.write(f'{test_case},{branching_factor},{method_name},{total_time},{cumulative_time},{per_call_time}\n')

    @staticmethod
    def get_running_times(st: pstats.Stats) -> List[Tuple[str, float, float, float]]:
        ps = st.strip_dirs().stats
        def is_heap_method(k):
            return 'heap' in k[2] or 'create_encoding' in k[2] or \
                   ('dway_heap.py' in k and ('top' in k[2] or 'insert' in k[2] or
                                             '_push_down' in k[2] or '_bubble_up' in k[2] or
                                             '_highest_priority_child_index' in k[2]))
        keys = list(filter(is_heap_method, ps.keys()))
        return [(key[2], ps[key][2], ps[key][3], ps[key][3] / ps[key][1]) for key in keys]

    def test_profile_huffman(self) -> None:
        with open(HuffmanProfile.OutputFileName, 'w') as f:
            HuffmanProfile.write_header(f)
            for test_case, (file_names, read_func, runs) in HuffmanProfile.TestCases.items():
                file_contents = [read_func(file_name) for file_name in file_names]
                for _ in range(runs):
                    for b in HuffmanProfile.BranchingFactors:
                        pro = cProfile.Profile()
                        for file_content in file_contents:
                            pro.runcall(create_encoding, file_content, b)
                        st = pstats.Stats(pro)
                        for method_name, total_time, cumulative_time, per_call_time in HuffmanProfile.get_running_times(st):
                            HuffmanProfile.write_row(f, test_case, b, method_name, total_time, cumulative_time, per_call_time)

# Ejecuta las pruebas de perfilado
suite = unittest.TestLoader().loadTestsFromTestCase(HuffmanProfile)
unittest.TextTestRunner(verbosity=2).run(suite)

            

#### Ejercicios

1 . Dado el siguiente conjunto de caracteres y sus frecuencias:
- A: 0.4
- B: 0.3
- C: 0.2
- D: 0.1

Calcula manualmente el árbol de Huffman siguiendo el procedimiento:

- Ordena los caracteres por frecuencia.
- Combina los dos nodos de menor frecuencia para formar un nodo padre.
- Repite el proceso hasta que quede un único nodo.

Dibuja el árbol resultante y escribe el código binario asignado a cada carácter (recuerda: 0 para rama izquierda y 1 para derecha).

2 . Analiza la complejidad temporal del algoritmo de Huffman, considerando:

- La construcción del mapa de frecuencias.
- La inserción y extracción en el heap.
- La construcción de la tabla de códigos mediante la función recursiva.

Compara la complejidad teórica con la práctica, discutiendo posibles cuellos de botella y cómo podría optimizarse el algoritmo para textos muy largos o con un gran número de caracteres únicos.

3 . Explica el proceso de generación del árbol de Huffman a partir de una tabla de frecuencias, detallando el rol de las funciones `_create_frequency_table`, `_frequency_table_to_heap` y `_heap_to_tree`.  Discute la importancia de invertir las frecuencias (utilizando valores negativos) al trabajar con un heap máximo y cómo se combinan los nodos para formar el árbol final.

4 . Investiga cómo varía el rendimiento (en tiempo de ejecución y consumo de memoria) del algoritmo de Huffman al modificar el factor de ramificación del heap d-ario.  
- **Tareas:**  
  - Utilizar el código base para generar codificaciones Huffman con diferentes valores de _branching_factor_ (por ejemplo, desde 2 hasta 23).  
  - Realizar pruebas de rendimiento y comparar los resultados obtenidos mediante perfilado.  
  - Interpretar y explicar cómo y por qué el factor de ramificación afecta el número de comparaciones, la profundidad del heap y la eficiencia global del algoritmo.

5 . Amplia la implementación actual para incluir funciones de compresión y descompresión de textos (y, opcionalmente, de imágenes).  
- **Tareas:**  
  - Diseñar e integrar funciones que permitan convertir un texto en su representación comprimida utilizando la codificación generada por Huffman.  
  - Implementar el proceso inverso para recuperar el texto original a partir del código comprimido.  
  - Diseñar pruebas unitarias adicionales que verifiquen la integridad de la compresión/descompresión en distintos escenarios y tamaños de entrada.


6 . Identifica posibles cuellos de botella en la función que construye el árbol de Huffman y proponer mejoras.  
- **Tareas:**  
  - Analizar el rendimiento de las funciones `_push_down`, `_bubble_up` y `_highest_priority_child_index` en el contexto del heap d-ario.  
  - Utilizar herramientas de perfilado para determinar qué métodos consumen más tiempo y optimizarlos sin alterar la semántica del algoritmo.  
  - Justificar las modificaciones realizadas y evaluar el impacto de las optimizaciones en diferentes casos de prueba.

7 . Profundiza en la verificación de la consistencia y la correcta construcción del árbol de Huffman.  
- **Tareas:**  
  - Implementar casos de prueba adicionales que verifiquen las invariantes del heap y la validez del árbol (por ejemplo, asegurando que cada nodo interno cumpla la suma correcta de prioridades y que los símbolos se concatenen adecuadamente).  
  - Diseñar escenarios de prueba con datos sintéticos y reales (textos de distinta longitud y complejidad) para evaluar la robustez de las funciones de validación (`_validate` en ambos contextos).  
  - Documentar posibles fallos y proponer estrategias para mejorar la detección temprana de errores en la construcción del árbol.

8 . Consolida los resultados obtenidos a partir del perfilado en un reporte analítico detallado.  
- **Tareas:**  
  - Modificar y ampliar el script de perfilado para capturar métricas más detalladas (por ejemplo, tiempos de llamada, número de invocaciones y consumo de CPU) para las funciones clave del algoritmo.  
  - Exportar los datos a un archivo CSV y generar visualizaciones (gráficos) que ilustren el comportamiento de cada función en función del factor de ramificación y el tipo de dato comprimido.  
  - Elaborar un informe que incluya análisis comparativos, gráficos y conclusiones sobre el desempeño y las posibles mejoras del algoritmo.


In [ ]:
### Tus respuestas

#### Visualización y análisis de estadísticas de rendimiento para algoritmos de Huffman y Heap

In [ ]:
%matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
import math

# Rutas de los archivos de estadísticas
HUFFMAN_STATS = '/data/stats_huffman.csv'
HEAP_STATS = '/data/stats_heap.csv'
HEAP_MIXED_STATS = '/data/stats_heap_mixed.csv'
HEAPIFY_STATS = '/data/stats_heapify.csv'

def plot_test_case_stats(df: pd.DataFrame, test_case: str, time_field: str = 'cumulative_time'):
    """
    Grafica los datos de un caso de prueba específico, generando gráficos para cada método.
    
    Parámetros:
        df: DataFrame con los datos de estadísticas.
        test_case: Nombre del caso de prueba a graficar.
        time_field: Campo de tiempo a utilizar (por defecto 'cumulative_time').
    """
    # Filtra el DataFrame para obtener solo los datos del caso de prueba indicado
    df_test_case = df[df['test_case'] == test_case]
    # Obtiene los nombres únicos de métodos en el caso de prueba
    method_names = df_test_case['method_name'].unique()
    
    # Genera un gráfico para cada método
    for method_name in method_names:
        plot_method_stats(df_test_case, method_name, time_field)

def plot_method_stats(df: pd.DataFrame, method_name: str, time_field: str):
    """
    Genera un gráfico de caja (boxplot) para visualizar la distribución de tiempos de ejecución
    de un método específico, según distintos factores de ramificación.
    
    Parámetros:
        df: DataFrame filtrado para el caso de prueba.
        method_name: Nombre del método a graficar.
        time_field: Campo de tiempo a utilizar.
    """
    # Filtra el DataFrame para obtener solo los datos del método especificado
    df_method: pd.DataFrame = df[df['method_name'] == method_name]
    
    # Obtiene los factores de ramificación únicos
    branching_factors = df_method['branching_factor'].unique()
    # Agrupa los datos por cada factor de ramificación
    data = [df_method[df_method['branching_factor'] == b][time_field] for b in branching_factors]
    
    # Crea la figura y el eje para el gráfico
    fig, axe = plt.subplots()

    axe.set_title(method_name, fontsize=16)
    axe.set_xlabel('Factor de ramificación')
    axe.set_ylabel('Tiempo de ejecución')

    # Genera el gráfico de caja sin mostrar valores atípicos
    plt.boxplot(x=data, labels=branching_factors, showfliers=False)
    plt.show()

def plot_test_case_means(df: pd.DataFrame, test_case: str, time_field: str = 'cumulative_time'):
    """
    Grafica las medias de tiempo de ejecución para cada método en un caso de prueba,
    según el factor de ramificación.
    
    Parámetros:
        df: DataFrame con los datos de estadísticas.
        test_case: Nombre del caso de prueba a graficar.
        time_field: Campo de tiempo a utilizar (por defecto 'cumulative_time').
    """
    # Filtra el DataFrame para obtener solo los datos del caso de prueba indicado
    df_test_case = df[df['test_case'] == test_case]
    # Obtiene los nombres únicos de métodos en el caso de prueba
    method_names = df_test_case['method_name'].unique()
    
    # Genera un gráfico para cada método mostrando la media
    for method_name in method_names:
        plot_method_mean(df_test_case, method_name, time_field)

def plot_method_mean(df: pd.DataFrame, method_name: str, time_field: str):
    """
    Genera un gráfico de líneas que muestra la media de tiempos de ejecución de un método
    específico, agrupados por factor de ramificación.
    
    Parámetros:
        df: DataFrame filtrado para el caso de prueba.
        method_name: Nombre del método a graficar.
        time_field: Campo de tiempo a utilizar.
    """
    # Filtra el DataFrame para obtener solo los datos del método especificado
    df_method: pd.DataFrame = df[df['method_name'] == method_name]
    
    # Obtiene los factores de ramificación únicos
    branching_factors = df_method['branching_factor'].unique()
    # Agrupa los datos por factor de ramificación y calcula la media
    data = df_method.groupby('branching_factor').mean().reset_index()
    
    # Crea la figura y el eje para el gráfico
    fig, axe = plt.subplots()

    axe.set_title(method_name, fontsize=16)
    axe.set_xlabel('Factor de ramificación')
    axe.set_ylabel('Tiempo de ejecución')

    # Genera el gráfico de líneas con la media de tiempo de ejecución
    plt.plot(branching_factors, data[time_field])
    plt.show()


### Ejercicios


1. **Carga y exploración de datos**  
   - Crea un DataFrame a partir del archivo CSV de estadísticas (por ejemplo, el archivo de Huffman).  
   - Utiliza funciones propias de pandas para inspeccionar la estructura del DataFrame (por ejemplo, revisa la información de columnas y tipos de datos).  
   - Extrae y lista todos los casos de prueba presentes en el DataFrame.  
   - Reflexiona sobre la importancia de conocer la estructura y el contenido de los datos antes de analizarlos.

2. **Visualización de estadísticas para casos de prueba de texto**  
   - Emplea las funciones de visualización para generar gráficos de caja que muestren la distribución de tiempos de ejecución para el caso de prueba relacionado a texto, utilizando distintos campos como el tiempo por llamada y el tiempo acumulado.  
   - Crea gráficos de líneas que representen la media de los tiempos de ejecución para el mismo caso de prueba, variando entre campos como tiempo acumulado, por llamada y tiempo total.  
   - Analiza qué información adicional aporta cada tipo de gráfico y cómo pueden ayudar a identificar tendencias o anomalías en el rendimiento.

3. **Visualización de estadísticas para casos de prueba de imágenes**  
   - Utiliza las funciones de visualización para generar gráficos que muestren los tiempos de ejecución para el caso de prueba de imágenes, centrándote en los tiempos por llamada y los tiempos acumulados.  
   - Compara los resultados obtenidos con los del caso de prueba de texto y comenta posibles diferencias en el rendimiento y la variabilidad.

4. **Análisis de estadísticas relacionadas con el heap**  
   - Crea DataFrames a partir de los archivos CSV correspondientes a las estadísticas del heap, las estadísticas mixtas del heap y las estadísticas de heapify.  
   - Genera gráficos que muestren los tiempos de ejecución para el caso de prueba "heap", utilizando las funciones de visualización disponibles.  
   - Discute las diferencias en la presentación de datos entre los distintos archivos y qué información relevante se puede extraer sobre el comportamiento del heap.

5. **Cálculo y análisis del número de swaps en heapify**  
   - Estudia la función encargada de calcular el número de swaps durante la operación heapify.  
   - Analiza cómo varía el número de swaps en función del tamaño del heap (por ejemplo, para `n` igual a 100, 1000, 10000 y 100000) y el factor de ramificación.  
   - Reflexiona sobre el impacto del factor de ramificación en la eficiencia de la operación, teniendo en cuenta que el gráfico generado utiliza una escala logarítmica en el eje de los swaps.

6.   A partir de los gráficos y análisis previos, elabora un informe en el que se discuta la relación entre el factor de ramificación y el rendimiento en las operaciones del heap y la construcción del árbol de Huffman.  Considera aspectos como la variabilidad de los tiempos de ejecución, la media de los mismos y la influencia del tamaño del conjunto de datos en el número de swaps durante la operación heapify. Finalmente, reflexiona sobre cómo estos análisis pueden orientar futuras optimizaciones o ajustes en la implementación.


In [ ]:
## Tus respuestas